In [32]:
import requests
from bs4 import BeautifulSoup
import csv

# Base URL of the company list page
base_url = "https://nwba.nl"

# URL of the company list page
company_list_url = "https://nwba.nl/nl/leden"

# Send a GET request to the company list URL and get the response
response = requests.get(company_list_url)

# Parse the HTML content of the response using BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')

# Find all the <a> tags inside <div class="raxo-readmore">
company_links = soup.select('div.raxo-readmore a')

# Open a CSV file for writing
csv_file = open('data/netherlands_member.csv', 'w', newline='', encoding='utf-8')
csv_writer = csv.DictWriter(csv_file, fieldnames=['Company Name', 'Description', 'Address', 'City', 'Postal Code', 'Contact Person', 'Email'])
csv_writer.writeheader()

# Extract the company information
for link in company_links:
    # Get the URL of the company page
    company_url = base_url + link['href']
    
    # Send a GET request to the company URL and get the response
    company_response = requests.get(company_url)
    
    # Parse the HTML content of the company response using BeautifulSoup
    company_soup = BeautifulSoup(company_response.content, 'html.parser')
    
    # Extract company description
    company_description = company_soup.find('p', class_='content-description')
    if company_description:
        company_description = company_description.text.strip()
    else:
        company_description = None
    
    # Extract company address details
    address_fields = company_soup.select('ul.fields-container li')
    address_info = {field.find('span', class_='field-label').text.strip(): field.find('span', class_='field-value').text.strip() for field in address_fields}
    
    # Extract address, city, and postal code
    company_name = address_info.get('Bedrijfsnaam:','')
    address = address_info.get('Adres:', '')
    city = address_info.get('Plaats:', '')
    postal_code = address_info.get('Postcode:', '')
    
    # Extract contact person and email
    contact_person = address_info.get('Contactpersoon:', '')
    email = address_info.get('E-mail:', '')
    
    # Write the data to the CSV file
    csv_writer.writerow({
        'Company Name': company_name,
        'Description': company_description,
        'Address': address,
        'City': city,
        'Postal Code': postal_code,
        'Contact Person': contact_person,
        'Email': email
    })

# Close the CSV file
csv_file.close()

In [35]:
# Load the CSV file
df = pd.read_csv('data/netherlands_member.csv')

# Find the index of the row with company name "Electriq"
electriq_index = df[df['Company Name'] == 'Electriq'].index.tolist()[0]

# Update the address details for "Electriq"
df.at[electriq_index, 'Address'] = 'De Stuwdam 33'
df.at[electriq_index, 'City'] = 'Amersfoort'
df.at[electriq_index, 'Postal Code'] = '3815KM'

# Combine Address, City, and Postal Code into one column
df['Full Address'] = df.apply(lambda row: f"{row['Address']}, {row['City']}, {row['Postal Code']}", axis=1)

# Save the updated DataFrame to the CSV file
df.to_csv('data/netherlands_member.csv', index=False)